In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore")

url_hourly = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/transformers_dataset.csv"
df_hourly = pd.read_csv(url_hourly,  sep=';', encoding='latin-1')

url_daily = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/daily_peak_transformers_dataset.csv"
df_daily = pd.read_csv(url_daily,  sep=';', encoding='latin-1')


In [ ]:

# df_count = df_daily.groupby('id').count()

# df_count = df_count.sort_values('datahora').tail(41).reset_index() # esses são os trafos com mais de 97,5% de linhas preenchidas
# trafos_escolhidos = df_count['id'].unique().tolist()
# df_filtrado = df_daily[df_daily['id'].isin(trafos_escolhidos)]

# fig_aparente = px.line(df_filtrado, x='datahora', y='S', color='id',
#                        title='Potência Aparente ao Longo do Tempo por Transformador',
#                        labels={'S': 'Potência Aparente (kVA)', 'Dia': 'Data'})

# fig_aparente.show()
# fig_aparente.write_html("Demanda ao longo do tempo - IQR.html")


#### Algoritmo de Aprendizado de Máquina - Pico Diário, 1 feature

In [9]:
# ========================================================================
# IMPORTAÇÕES E CONFIGURAÇÕES
# ========================================================================
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'

from math import sqrt
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import joblib

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import keras_tuner as kt

import matplotlib
matplotlib.use('Agg')

# Fixando seeds
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# ========================================================================
# FUNÇÕES AUXILIARES
# ========================================================================
def gerar_tabela_metricas_por_fold(trafo, modelo, fold_rmse, fold_mae):
    return pd.DataFrame({
        'Fold': [f'Fold {i+1}' for i in range(len(fold_rmse))],
        'Trafo': trafo,
        'Modelo': modelo,
        'RMSE': np.round(fold_rmse, 4),
        'MAE': np.round(fold_mae, 4)
    })

def plotar_ultimo_fold(datas, y_real, y_pred, trafo, modelo, freq):
    plt.figure(figsize=(14,6))
    plt.plot(datas, y_real, label='Real', color='blue')
    plt.plot(datas, y_pred, label=f'Previsto ({modelo})', linestyle='--', color='orange')
    plt.xlabel('Dia' if freq=='daily' else 'Hora')
    plt.ylabel('Potência Aparente (kVA)')
    plt.title(f'Previsão Último Fold - {modelo} ({trafo})')
    plt.legend()
    plt.tight_layout()
    os.makedirs('plots', exist_ok=True)
    plt.savefig(f'plots/PLOT_{modelo}_{trafo}_{freq}_ultimo_fold.pdf', dpi=300, bbox_inches='tight')
    plt.show()

def plotar_todos_folds(lista_datas, lista_reais, lista_previstos, trafo, modelo, eixo_label='Data'):
    datas_todas = pd.to_datetime(np.concatenate(lista_datas))
    reais_todos = np.concatenate(lista_reais)
    previstos_todos = np.concatenate(lista_previstos)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=datas_todas, y=reais_todos, mode='lines', name='Real', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=datas_todas, y=previstos_todos, mode='lines', name=f'Previsto ({modelo})', line=dict(color='orange', dash='dash')))
    fig.update_layout(title=f'Previsão em Todos os Folds - {trafo} ({modelo})',
                      xaxis_title=eixo_label, yaxis_title='Potência Aparente', hovermode='x unified')
    fig.show()

# ========================================================================
# HYPERPARAMS PARA RANDOMIZEDSEARCH
# ========================================================================
param_grids = {
    'SVR': {
        'C': [0.1, 1, 10, 100, 1000],
        'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
        'epsilon': [0.001, 0.01, 0.1, 0.5]
    },
    'RFR': {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 20, None],
        'min_samples_split': [2, 5, 10]
    },
    'GBR': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7]
    },
    'XGB': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3,5,7]
    },
    'LGBM': {
        'n_estimators': [50,100,200],
        'learning_rate': [0.01,0.05,0.1],
        'max_depth': [-1,3,5,7]
    }
}

# ========================================================================
# FUNÇÃO PRINCIPAL
# ========================================================================
def treinar_e_prever_modelo_auto(data, trafos_escolhidos, modelo, janela=None, epochs=20, batch_size=32, n_iter_search=10):
    resultados = []
    os.makedirs('modelos', exist_ok=True)
    data['datahora'] = pd.to_datetime(data['datahora'])
    
    delta = data['datahora'].diff().median()
    if delta >= pd.Timedelta('1D'):
        freq = 'daily'
        if janela is None: janela = 30
        eixo_label = 'Dia'
    else:
        freq = 'hourly'
        if janela is None: janela = 24*7
        eixo_label = 'Hora'
        # Filtra só 2023 e 2024
        data = data[(data['datahora'].dt.year >= 2023) & (data['datahora'].dt.year <= 2024)]

    for trafo in trafos_escolhidos:
        print(f"\n🔹 Treinando {modelo} para {trafo} ({freq})...")
        df = data[data['id']==trafo].copy()
        df = df[['datahora','S']].sort_values('datahora').dropna()
        
        # Janelas deslizantes
        X, y, datas_janela = [], [], []
        S_values = df['S'].values
        datas = df['datahora'].values
        for i in range(janela, len(df)):
            X.append(S_values[i-janela:i])
            y.append(S_values[i])
            datas_janela.append(datas[i])
        X, y = np.array(X), np.array(y)
        datas_janela = np.array(datas_janela)
        if modelo in ['LSTM','CNN','CNN_LSTM']: X = X.reshape((X.shape[0], X.shape[1],1))
        else: X = X.reshape((X.shape[0], -1))

        tscv = TimeSeriesSplit(n_splits=5)
        fold_rmse, fold_mae = [], []
        lista_datas, lista_reais, lista_previstos = [], [], []

        for fold_idx, (train_idx, test_idx) in enumerate(tscv.split(X)):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # ====================
            # MODELOS COM HYPERPARAMS
            # ====================
            if modelo in param_grids.keys(): # scikit-learn
                base_model = {
                    'SVR': SVR(),
                    'RFR': RandomForestRegressor(random_state=42, n_jobs=-1),
                    'GBR': GradientBoostingRegressor(random_state=42),
                    'XGB': XGBRegressor(random_state=42, objective='reg:squarederror'),
                    'LGBM': LGBMRegressor(random_state=42)
                }[modelo]
                search = RandomizedSearchCV(base_model, param_distributions=param_grids[modelo],
                                            n_iter=n_iter_search, scoring='neg_mean_squared_error',
                                            cv=TimeSeriesSplit(n_splits=3), n_jobs=-1, random_state=42)
                search.fit(X_train, y_train)
                best_model = search.best_estimator_
                y_pred = best_model.predict(X_test)

            else: # Redes neurais
                def build_nn(hp):
                    model = Sequential()
                    if modelo=='LSTM':
                        model.add(LSTM(units=hp.Int('units1', 20, 100, step=20), return_sequences=True, input_shape=(X_train.shape[1],1)))
                        model.add(LSTM(units=hp.Int('units2',20,100, step=20)))
                        model.add(Dense(1))
                        model.compile(optimizer='adam', loss='mse')
                    elif modelo=='CNN':
                        model.add(Conv1D(filters=hp.Int('filters',32,128,step=32), kernel_size=2, activation='relu', input_shape=(X_train.shape[1],1)))
                        model.add(MaxPooling1D(2))
                        model.add(Flatten())
                        model.add(Dense(hp.Int('dense',10,50,step=10), activation='relu'))
                        model.add(Dense(1))
                        model.compile(optimizer='adam', loss='mse')
                    elif modelo=='CNN_LSTM':
                        model.add(Conv1D(filters=hp.Int('filters',32,128,step=32), kernel_size=2, activation='relu', input_shape=(X_train.shape[1],1)))
                        model.add(MaxPooling1D(2))
                        model.add(LSTM(units=hp.Int('units1',20,100,step=20), return_sequences=True))
                        model.add(LSTM(units=hp.Int('units2',20,100,step=20)))
                        model.add(Dense(1))
                        model.compile(optimizer='adam', loss='mse')
                    return model
                tuner = kt.RandomSearch(build_nn, objective='val_loss', max_trials=5, overwrite=True)
                tuner.search(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
                best_model = tuner.get_best_models(num_models=1)[0]
                y_pred = best_model.predict(X_test).flatten()

            # ====================
            # MÉTRICAS
            # ====================
            rmse = sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            fold_rmse.append(rmse)
            fold_mae.append(mae)
            datas_finais = datas_janela[test_idx]
            lista_datas.append(datas_finais)
            lista_reais.append(y_test)
            lista_previstos.append(y_pred)

        # Plot do último fold
        plotar_ultimo_fold(lista_datas[-1], lista_reais[-1], lista_previstos[-1], trafo, modelo, freq)

        # Salva modelo final
        modelo_path = f"modelos/{modelo}_{trafo}.{'h5' if modelo in ['LSTM','CNN','CNN_LSTM'] else 'pkl'}"
        if modelo in ['LSTM','CNN','CNN_LSTM']:
            best_model.save(modelo_path)
        else:
            joblib.dump(best_model, modelo_path)
        print(f"✅ Modelo salvo em: {modelo_path}")

        # Métricas
        df_metricas = gerar_tabela_metricas_por_fold(trafo, modelo, fold_rmse, fold_mae)
        print(df_metricas)

        # Plot todos folds
        plotar_todos_folds(lista_datas, lista_reais, lista_previstos, trafo, modelo, eixo_label=eixo_label)

        resultados.append({
            'Trafo': trafo,
            'Modelo': modelo,
            'RMSE Médio': np.round(np.mean(fold_rmse),4),
            'MAE Médio': np.round(np.mean(fold_mae),4),
            'RMSE Último Fold': np.round(fold_rmse[-1],4),
            'MAE Último Fold': np.round(fold_mae[-1],4)
        })

    return pd.DataFrame(resultados)

# ========================================================================
# USO
# ========================================================================
trafos = ['T1']
resultados_SVR_daily = treinar_e_prever_modelo_auto(df_daily, trafos, modelo='SVR', janela=30, n_iter_search=10)
resultados_SVR_hourly = treinar_e_prever_modelo_auto(df_hourly, trafos, modelo='SVR', janela=168, n_iter_search=10)



🔹 Treinando SVR para T1 (daily)...
✅ Modelo salvo em: modelos/SVR_T1.pkl
     Fold Trafo Modelo  RMSE  MAE
0  Fold 1    T1    SVR  0.05 0.03
1  Fold 2    T1    SVR  0.04 0.03
2  Fold 3    T1    SVR  0.04 0.03
3  Fold 4    T1    SVR  0.05 0.03
4  Fold 5    T1    SVR  0.05 0.04

🔹 Treinando SVR para T1 (hourly)...
✅ Modelo salvo em: modelos/SVR_T1.pkl
     Fold Trafo Modelo  RMSE  MAE
0  Fold 1    T1    SVR  0.07 0.05
1  Fold 2    T1    SVR  0.07 0.05
2  Fold 3    T1    SVR  0.06 0.04
3  Fold 4    T1    SVR  0.06 0.04
4  Fold 5    T1    SVR  0.06 0.05


In [ ]:
import os
from tensorflow.keras.models import load_model
import joblib

# Lista de transformadores e modelos
trafos = ['T1', 'T2']
modelos = ['SVR', 'XGB', 'LGBM', 'GBR', 'LSTM', 'CNN', 'CNN_LSTM']

# Diretório base onde estão os modelos
base_dir = r"G:\Meu Drive\Estudos\Mestrado\Github\masters\modelos"

# Dicionário para armazenar os modelos carregados
modelos_carregados = {}

for trafo in trafos:
    for modelo in modelos:
        # Define a extensão correta
        extensao = "h5" if modelo in ["LSTM", "CNN", "CNN_LSTM"] else "pkl"
        caminho = os.path.join(base_dir, f"{modelo}_{trafo}.{extensao}")
        
        # Inicializa a chave no dicionário
        modelos_carregados[(modelo, trafo)] = None
        
        # Carrega apenas se o arquivo existir
        if os.path.exists(caminho):
            if modelo in ["LSTM", "CNN", "CNN_LSTM"]:
                modelos_carregados[(modelo, trafo)] = load_model(caminho)
            else:
                modelos_carregados[(modelo, trafo)] = joblib.load(caminho)
            print(f"✅ Modelo carregado: {caminho}")
        else:
            print(f"⚠️ Arquivo não encontrado, ignorando: {caminho}")


In [ ]:
# Vamos analisar seus dados originais para entender a estrutura
print("=== ANALISANDO DADOS ORIGINAIS ===")
print(f"Shape do df_daily: {df_daily.shape}")
print(f"Colunas: {df_daily.columns.tolist()}")
print(f"Tipos de dados:\n{df_daily.dtypes}")

# Ver um exemplo dos dados do T1
print("\n=== DADOS DO TRAFO T1 ===")
df_t1 = df_daily[df_daily['id'] == 'T1'].copy()
print(f"Quantidade de dados T1: {len(df_t1)}")
print(f"Período: {df_t1['datahora'].min()} até {df_t1['datahora'].max()}")
print(f"Primeiras linhas T1:")
print(df_t1.head())

# Ver a distribuição da variável target 'S'
print(f"\nEstatísticas da Potência Aparente (S):")
print(df_t1['S'].describe())

# TREINNAMENTO ML

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")

url_hourly = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/transformers_dataset.csv"
df_hourly = pd.read_csv(url_hourly,  sep=';', encoding='latin-1')

url_daily = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/daily_peak_transformers_dataset.csv"
df_daily = pd.read_csv(url_daily,  sep=';', encoding='latin-1')


In [ ]:
# ========================================================================
# IMPORTAÇÕES E CONFIGURAÇÕES
# ========================================================================
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'

from math import sqrt
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import joblib

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

import matplotlib
matplotlib.use('Agg')

# Fixando seeds
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# ========================================================================
# FUNÇÕES AUXILIARES
# ========================================================================
def gerar_tabela_metricas_por_fold(trafo, modelo, fold_rmse, fold_mae):
    return pd.DataFrame({
        'Fold': [f'Fold {i+1}' for i in range(len(fold_rmse))],
        'Trafo': trafo,
        'Modelo': modelo,
        'RMSE': np.round(fold_rmse, 4),
        'MAE': np.round(fold_mae, 4)
    })

def plotar_ultimo_fold(datas, y_real, y_pred, trafo, modelo, freq):
    plt.figure(figsize=(14,6))
    plt.plot(datas, y_real, label='Real', color='blue')
    plt.plot(datas, y_pred, label=f'Previsto ({modelo})', linestyle='--', color='orange')
    plt.xlabel('Dia' if freq=='daily' else 'Hora')
    plt.ylabel('Potência Aparente (kVA)')
    plt.title(f'Previsão Último Fold - {modelo} ({trafo})')
    plt.legend()
    plt.tight_layout()
    os.makedirs('plots', exist_ok=True)
    plt.savefig(f'plots/PLOT_{modelo}_{trafo}_{freq}_ultimo_fold.pdf', dpi=300, bbox_inches='tight')
    plt.show()

def plotar_todos_folds(lista_datas, lista_reais, lista_previstos, trafo, modelo, eixo_label='Data'):
    datas_todas = pd.to_datetime(np.concatenate(lista_datas))
    reais_todos = np.concatenate(lista_reais)
    previstos_todos = np.concatenate(lista_previstos)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=datas_todas, y=reais_todos, mode='lines', name='Real', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=datas_todas, y=previstos_todos, mode='lines', name=f'Previsto ({modelo})', line=dict(color='orange', dash='dash')))
    fig.update_layout(title=f'Previsão em Todos os Folds - {trafo} ({modelo})',
                      xaxis_title=eixo_label, yaxis_title='Potência Aparente', hovermode='x unified')
    fig.show()

# ========================================================================
# HYPERPARAMS OTIMIZADOS
# ========================================================================
param_grids_base = {
    'SVR': {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.01, 0.1],
        'epsilon': [0.01, 0.1, 0.5]
    },
    'RFR': {
        'n_estimators': [50, 100, 150],
        'max_depth': [5, 10, 15],
        'min_samples_split': [2, 5]
    },
    'GBR': {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.05, 0.1],
        'max_depth': [3, 5]
    },
    'XGB': {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.05, 0.1],
        'max_depth': [3, 5]
    },
    'LGBM': {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.05, 0.1],
        'max_depth': [5, 7]
    }
}

def get_param_grids_por_freq(freq, modelo):
    """Retorna grid de parâmetros baseado na frequência dos dados"""
    base_params = param_grids_base[modelo].copy()
    
    if freq == 'hourly':  # Dados horários - grids mais simples
        if modelo == 'SVR':
            base_params['C'] = [1, 10]
            base_params['gamma'] = ['scale', 0.01]
            base_params['epsilon'] = [0.1]
        elif modelo in ['RFR', 'GBR', 'XGB', 'LGBM']:
            base_params['n_estimators'] = [50, 100]  # Menos árvores
    return base_params

# ========================================================================
# FUNÇÃO PRINCIPAL OTIMIZADA
# ========================================================================
def treinar_e_prever_modelo_auto_otimizado(data, trafos_escolhidos, modelo, janela=None, epochs=15, batch_size=32, n_iter_search=5):
    """Versão otimizada da função principal"""
    resultados = []
    os.makedirs('modelos', exist_ok=True)
    data['datahora'] = pd.to_datetime(data['datahora'])
    
    # Determina frequência e configurações
    delta = data['datahora'].diff().median()
    if delta >= pd.Timedelta('1D'):
        freq = 'daily'
        if janela is None: janela = 30
        eixo_label = 'Dia'
    else:
        freq = 'hourly'
        if janela is None: janela = 24*3  # 3 dias em vez de 7
        eixo_label = 'Hora'
        # Filtra só 2023 e 2024 de forma mais eficiente
        data = data[data['datahora'].dt.year.isin([2023, 2024])].copy()

    # PRÉ-COMPUTAÇÃO: Criar dicionário com dados de cada trafo
    dados_trafos = {}
    for trafo in trafos_escolhidos:
        df_trafo = data[data['id']==trafo][['datahora','S']].sort_values('datahora').dropna()
        dados_trafos[trafo] = {
            'S_values': df_trafo['S'].values,
            'datas': df_trafo['datahora'].values
        }

    # Configurações específicas por frequência
    if freq == 'hourly':
        n_iter_search = max(2, n_iter_search // 2)  # Menos iterações
        epochs = min(10, epochs)  # Menos epochs
        batch_size = min(64, batch_size)  # Batch menor

    for trafo in trafos_escolhidos:
        print(f"\n🔹 Treinando {modelo} para {trafo} ({freq})...")
        
        # Uso dos dados pré-computados
        S_values = dados_trafos[trafo]['S_values']
        datas = dados_trafos[trafo]['datas']
        
        # Criação MAIS EFICIENTE das janelas usando sliding_window_view
        n_samples = len(S_values) - janela
        if n_samples <= 0:
            print(f"⚠️ Dados insuficientes para {trafo}. Pulando...")
            continue
            
        X = np.lib.stride_tricks.sliding_window_view(S_values, janela)[:n_samples]
        y = S_values[janela:janela + n_samples]
        datas_janela = datas[janela:janela + n_samples]
        
        # Reshape para modelos
        if modelo in ['LSTM','CNN','CNN_LSTM']: 
            X = X.reshape((X.shape[0], X.shape[1], 1))
        else: 
            X = X.reshape((X.shape[0], -1))

        tscv = TimeSeriesSplit(n_splits=5)
        fold_rmse, fold_mae = [], []
        lista_datas, lista_reais, lista_previstos = [], [], []

        for fold_idx, (train_idx, test_idx) in enumerate(tscv.split(X)):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # ====================
            # MODELOS COM HYPERPARAMS OTIMIZADOS
            # ====================
            if modelo in param_grids_base.keys(): # scikit-learn
                param_grid_freq = get_param_grids_por_freq(freq, modelo)
                
                base_model = {
                    'SVR': SVR(),
                    'RFR': RandomForestRegressor(random_state=42, n_jobs=-1),
                    'GBR': GradientBoostingRegressor(random_state=42),
                    'XGB': XGBRegressor(random_state=42, objective='reg:squarederror'),
                    'LGBM': LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1)  # Silencioso
                }[modelo]
                
                # RandomizedSearch mais rápido
                search = RandomizedSearchCV(
                    base_model, 
                    param_distributions=param_grid_freq,
                    n_iter=n_iter_search, 
                    scoring='neg_mean_squared_error',
                    cv=TimeSeriesSplit(n_splits=2),  # Menos folds de validação
                    n_jobs=1 if modelo in ['SVR'] else -1,  # SVR não paraleliza bem
                    random_state=42,
                    verbose=0  # Silencioso
                )
                search.fit(X_train, y_train)
                best_model = search.best_estimator_
                y_pred = best_model.predict(X_test)

            else: # Redes neurais
                def build_nn_otimizada(hp):
                    model = Sequential()
                    
                    # Configurações baseadas na frequência
                    units_multiplier = 0.7 if freq == 'hourly' else 1
                    
                    if modelo == 'LSTM':
                        model.add(LSTM(
                            units=hp.Int('units1', 
                                       int(20*units_multiplier), 
                                       int(80*units_multiplier), 
                                       step=20), 
                            input_shape=(X_train.shape[1], 1)
                        ))
                        model.add(Dense(1))
                    elif modelo == 'CNN':
                        model.add(Conv1D(
                            filters=hp.Int('filters', 32, 64, step=32), 
                            kernel_size=2, 
                            activation='relu', 
                            input_shape=(X_train.shape[1], 1)
                        ))
                        model.add(MaxPooling1D(2))
                        model.add(Flatten())
                        model.add(Dense(1))
                    elif modelo == 'CNN_LSTM':
                        model.add(Conv1D(
                            filters=32, 
                            kernel_size=2, 
                            activation='relu', 
                            input_shape=(X_train.shape[1], 1)
                        ))
                        model.add(MaxPooling1D(2))
                        model.add(LSTM(
                            units=hp.Int('units1', 20, 60, step=20)
                        ))
                        model.add(Dense(1))
                    
                    model.compile(optimizer='adam', loss='mse')
                    return model
                
                # Early stopping para redes neurais
                early_stop = EarlyStopping(
                    monitor='val_loss', 
                    patience=3, 
                    restore_best_weights=True
                )
                
                tuner = kt.RandomSearch(
                    build_nn_otimizada,
                    objective='val_loss',
                    max_trials=3,  # Menos trials
                    overwrite=True,
                    executions_per_trial=1,
                    directory='keras_tuner',
                    project_name=f'{modelo}_{trafo}_{freq}'
                )
                
                tuner.search(
                    X_train, y_train, 
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.2,
                    callbacks=[early_stop],
                    verbose=0
                )
                
                best_model = tuner.get_best_models(num_models=1)[0]
                y_pred = best_model.predict(X_test).flatten()

            # ====================
            # MÉTRICAS
            # ====================
            rmse = sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            fold_rmse.append(rmse)
            fold_mae.append(mae)
            datas_finais = datas_janela[test_idx]
            lista_datas.append(datas_finais)
            lista_reais.append(y_test)
            lista_previstos.append(y_pred)
            
            print(f"   Fold {fold_idx+1}: RMSE={rmse:.4f}, MAE={mae:.4f}")

        # Plot do último fold
        plotar_ultimo_fold(lista_datas[-1], lista_reais[-1], lista_previstos[-1], trafo, modelo, freq)

        # Salva modelo final
        modelo_path = f"modelos/{modelo}_{trafo}_{freq}.{'h5' if modelo in ['LSTM','CNN','CNN_LSTM'] else 'pkl'}"
        if modelo in ['LSTM','CNN','CNN_LSTM']:
            best_model.save(modelo_path)
        else:
            joblib.dump(best_model, modelo_path)
        print(f"✅ Modelo salvo em: {modelo_path}")

        # Métricas
        df_metricas = gerar_tabela_metricas_por_fold(trafo, modelo, fold_rmse, fold_mae)
        print(df_metricas)

        # Plot todos folds
        plotar_todos_folds(lista_datas, lista_reais, lista_previstos, trafo, modelo, eixo_label=eixo_label)

        resultados.append({
            'Trafo': trafo,
            'Modelo': modelo,
            'Frequência': freq,
            'RMSE Médio': np.round(np.mean(fold_rmse),4),
            'MAE Médio': np.round(np.mean(fold_mae),4),
            'RMSE Último Fold': np.round(fold_rmse[-1],4),
            'MAE Último Fold': np.round(fold_mae[-1],4)
        })

    return pd.DataFrame(resultados)

# ========================================================================
# USO OTIMIZADO
# ========================================================================
trafos = ['T1']

print("🚀 Iniciando treinamento com configurações otimizadas...")

# Para dados diários - configurações normais
print("\n📊 Treinando com dados DIÁRIOS (configuração padrão)...")

resultados_SVR_daily = treinar_e_prever_modelo_auto_otimizado(
    df_daily, 
    trafos, 
    'SVR', 
    janela=30,
    n_iter_search=5,
    epochs=15
)

print("\n✅ Todos os treinamentos concluídos!")
print("\nResultados Horários:")
print(resultados_SVR_hourly)
print("\nResultados Diários:")
print(resultados_SVR_daily)


🚀 Iniciando treinamento com configurações otimizadas...

📊 Treinando com dados DIÁRIOS (configuração padrão)...

🔹 Treinando SVR para T1 (daily)...
   Fold 1: RMSE=0.0473, MAE=0.0313
   Fold 2: RMSE=0.0413, MAE=0.0294
   Fold 3: RMSE=0.0362, MAE=0.0268
   Fold 4: RMSE=0.0477, MAE=0.0326
   Fold 5: RMSE=0.0527, MAE=0.0398
✅ Modelo salvo em: modelos/SVR_T1_daily.pkl
     Fold Trafo Modelo  RMSE  MAE
0  Fold 1    T1    SVR  0.05 0.03
1  Fold 2    T1    SVR  0.04 0.03
2  Fold 3    T1    SVR  0.04 0.03
3  Fold 4    T1    SVR  0.05 0.03
4  Fold 5    T1    SVR  0.05 0.04

✅ Todos os treinamentos concluídos!

Resultados Horários:
  Trafo Modelo  RMSE Médio  MAE Médio  RMSE Último Fold  MAE Último Fold
0    T1    SVR        0.06       0.05              0.06             0.05

Resultados Diários:
  Trafo Modelo Frequência  RMSE Médio  MAE Médio  RMSE Último Fold  \
0    T1    SVR      daily        0.04       0.03              0.05   

   MAE Último Fold  
0             0.04  


In [14]:
# Teste rápido com dados horários
resultados = treinar_e_prever_modelo_auto_otimizado(
    df_hourly, 
    ['T1'], 
    'LGBM',  # Mais rápido que SVR
    janela=24*3, 
    n_iter_search=2,
    epochs=8
)


🔹 Treinando LGBM para T1 (hourly)...
   Fold 1: RMSE=0.0509, MAE=0.0358
   Fold 2: RMSE=0.0609, MAE=0.0439
   Fold 3: RMSE=0.0505, MAE=0.0352
   Fold 4: RMSE=0.0495, MAE=0.0332
   Fold 5: RMSE=0.0576, MAE=0.0392
✅ Modelo salvo em: modelos/LGBM_T1_hourly.pkl
     Fold Trafo Modelo  RMSE  MAE
0  Fold 1    T1   LGBM  0.05 0.04
1  Fold 2    T1   LGBM  0.06 0.04
2  Fold 3    T1   LGBM  0.05 0.04
3  Fold 4    T1   LGBM  0.05 0.03
4  Fold 5    T1   LGBM  0.06 0.04


# CRIANDO FEATURES SAZONAIS

### Diário, multi-feature

In [ ]:
def criar_features_daily(df_daily):
    df_featured_daily = df_daily.copy()
        
    # Features para dados diários
    df_featured_daily['datahora'] = pd.to_datetime(df_featured_daily['datahora'])
    df_featured_daily = df_featured_daily.sort_values(['id', 'datahora'])
    
    df_featured_daily['day_of_week'] = df_featured_daily['datahora'].dt.dayofweek
    df_featured_daily['month'] = df_featured_daily['datahora'].dt.month
    df_featured_daily['year'] = df_featured_daily['datahora'].dt.year
    df_featured_daily['is_weekend'] = (df_featured_daily['day_of_week'] >= 5).astype(int)
    
    # Features cíclicas diárias
    df_featured_daily['sin_day_of_week'] = np.sin(2 * np.pi * df_featured_daily['day_of_week'] / 7)
    df_featured_daily['cos_day_of_week'] = np.cos(2 * np.pi * df_featured_daily['day_of_week'] / 7)
    df_featured_daily['sin_month'] = np.sin(2 * np.pi * df_featured_daily['month'] / 12)
    df_featured_daily['cos_month'] = np.cos(2 * np.pi * df_featured_daily['month'] / 12)
    
    # Lags diários (sazonais)
    for lag in [1, 7, 30, 365]:  # 1 dia, 1 semana, 1 mês, 1 ano
        df_featured_daily[f'S{lag}'] = df_featured_daily.groupby('id')['S'].shift(lag)
    
    # Médias móveis diárias
    for window in [7, 30, 90]:  # 1 semana, 1 mês, 3 meses
        df_featured_daily[f'S_rolling_mean_{window}'] = df_featured_daily.groupby('id')['S'].transform(
            lambda x: x.rolling(window, min_periods=1).mean())
        
    return df_featured_daily

print("\nCriando features de engenharia de tempo para dados diários...")
daily_features = criar_features_daily(df_daily)
daily_features
# data_com_features.dropna(inplace=True)
print("✅ Features diárias criadas com sucesso.")


### Hora em Hora, multi-feature

In [ ]:
def criar_features_hourly(df_hourly):
    # """Cria features específicas para cada modo"""    
    df_featured_hourly = df_hourly.copy()

    # Features para dados horários
    df_featured_hourly['datahora'] = pd.to_datetime(df_featured_hourly['datahora'])
    df_featured_hourly = df_featured_hourly.sort_values(['id', 'datahora'])
    
    df_featured_hourly['hour'] = df_featured_hourly['datahora'].dt.hour
    df_featured_hourly['day_of_week'] = df_featured_hourly['datahora'].dt.dayofweek
    df_featured_hourly['is_weekend'] = (df_featured_hourly['day_of_week'] >= 5).astype(int)
    
    # Features cíclicas horárias
    df_featured_hourly['sin_hour'] = np.sin(2 * np.pi * df_featured_hourly['hour'] / 24)
    df_featured_hourly['cos_hour'] = np.cos(2 * np.pi * df_featured_hourly['hour'] / 24)
    
    # Lags horários
    for lag in [1, 24, 168]:  # 1h, 1 dia, 1 semana
        df_featured_hourly[f'S_lag_{lag}'] = df_featured_hourly.groupby('id')['S'].shift(lag)
    
    # Médias móveis horárias
    df_featured_hourly['S_rolling_mean_24'] = df_featured_hourly.groupby('id')['S'].transform(
        lambda x: x.rolling(24, min_periods=1).mean())
        
        
    return df_featured_hourly

print("\nCriando features de engenharia de tempo para dados horários...")
hourly_features = criar_features_hourly(df_hourly)

# data_com_features.dropna(inplace=True)
print("✅ Features horárias criadas com sucesso.")


### TREINANDO MULTI-FEATURE

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt


# ========================================================================
# FUNÇÃO PRINCIPAL DE TREINAMENTO E PREVISÃO (ATUALIZADA)
# ========================================================================
def treinar_e_prever_modelo_ft(data, trafos_escolhidos, modelo, janela, epochs=20, batch_size=32):
    """
    Função para treinar e prever modelos de séries temporais com múltiplas features.
    
    Parâmetros:
        data (DataFrame): dataframe com colunas ['id', 'datahora', 'S'] + features extras
        trafos_escolhidos (list): lista de transformadores a treinar
        modelo (str): nome do modelo ('SVR', 'RFR', 'GBR', 'XGB', 'LGBM', 'LSTM', 'CNN', 'CNN_LSTM')
        janela (int): tamanho da janela deslizante (número de passos anteriores)
        epochs (int): número de épocas (para redes neurais)
        batch_size (int): tamanho do batch (para redes neurais)
    
    Retorna:
        DataFrame com as métricas médias por trafo.
    """
    resultados = []
    os.makedirs('modelos', exist_ok=True)

    for trafo in trafos_escolhidos:
        print(f"\n🔹 Treinando {modelo} para {trafo}...")

        # ======================
        # 1. Preparação dos dados
        # ======================
        df = data[data['id'] == trafo].copy()
        df = df.sort_values('datahora').dropna().reset_index(drop=True)

        # Seleciona todas as colunas numéricas exceto 'id', 'datahora' e 'S'
        feature_cols = [c for c in df.columns if c not in ['id', 'datahora', 'S']]
        target_col = 'S'

        X_raw = df[feature_cols].values
        y_raw = df[target_col].values
        datas = df['datahora'].values

        # Cria janelas deslizantes
        X, y, datas_janela = [], [], []
        for i in range(janela, len(df)):
            X.append(X_raw[i-janela:i])   # entrada: janela com features
            y.append(y_raw[i])            # saída: valor atual
            datas_janela.append(datas[i]) # datas para plot

        X, y = np.array(X), np.array(y)
        datas_janela = np.array(datas_janela)

        # Ajuste de shape
        if modelo in ['LSTM', 'CNN', 'CNN_LSTM']:
            X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))  # (amostras, timesteps, features)
        else:
            X = X.reshape(X.shape[0], -1)  # achata janelas + features

        # ======================
        # 2. Cross-validation temporal
        # ======================
        tscv = TimeSeriesSplit(n_splits=5)
        fold_rmse, fold_mae = [], []
        lista_datas, lista_reais, lista_previstos = [], [], []

        for fold_idx, (train_idx, test_idx) in enumerate(tscv.split(X)):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # ======================
            # 3. Inicializa modelo
            # ======================
            if modelo == 'SVR':
                regressor = SVR(kernel='rbf', C=100, gamma=0.001, epsilon=0.01)
            elif modelo == 'RFR':
                regressor = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)
            elif modelo == 'GBR':
                regressor = GradientBoostingRegressor(n_estimators=100, random_state=42)
            elif modelo == 'LGBM':
                regressor = LGBMRegressor(n_estimators=100, random_state=42)
            elif modelo == 'XGB':
                regressor = XGBRegressor(n_estimators=100, random_state=42, objective='reg:squarederror')
            elif modelo == 'LSTM':
                regressor = Sequential([
                    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
                    LSTM(50),
                    Dense(1)
                ])
                regressor.compile(optimizer='adam', loss='mse')
            elif modelo == 'CNN':
                regressor = Sequential([
                    Conv1D(64, 2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
                    MaxPooling1D(2),
                    Flatten(),
                    Dense(50, activation='relu'),
                    Dense(1)
                ])
                regressor.compile(optimizer='adam', loss='mse')
            elif modelo == 'CNN_LSTM':
                regressor = Sequential([
                    Conv1D(64, 2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
                    MaxPooling1D(2),
                    LSTM(50, return_sequences=True),
                    LSTM(50),
                    Dense(1)
                ])
                regressor.compile(optimizer='adam', loss='mse')

            # ======================
            # 4. Treinamento
            # ======================
            if modelo in ['LSTM', 'CNN', 'CNN_LSTM']:
                regressor.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
                y_pred = regressor.predict(X_test)
            else:
                regressor.fit(X_train, y_train)
                y_pred = regressor.predict(X_test)

            # ======================
            # 5. Avaliação
            # ======================
            rmse = sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            fold_rmse.append(rmse)
            fold_mae.append(mae)

            datas_finais = datas_janela[test_idx]
            lista_datas.append(datas_finais)
            lista_reais.append(y_test)
            lista_previstos.append(y_pred)

            if fold_idx == 4:
                plotar_resultados(datas_finais, y_test, y_pred, trafo, modelo)

        # ======================
        # 6. Salva modelo final
        # ======================
        modelo_path = f"modelos/{modelo}_{trafo}.{'h5' if modelo in ['LSTM', 'CNN', 'CNN_LSTM'] else 'pkl'}"
        if modelo in ['LSTM', 'CNN', 'CNN_LSTM']:
            regressor.save(modelo_path)
        else:
            joblib.dump(regressor, modelo_path)
        print(f"✅ Modelo salvo em: {modelo_path}")

        # ======================
        # 7. Métricas e plots
        # ======================
        df_metricas = gerar_tabela_metricas_por_fold(trafo, modelo, fold_rmse, fold_mae)
        print(df_metricas)

        plotar_todos_folds(lista_datas, lista_reais, lista_previstos, trafo, modelo)

        resultados.append({
            'Trafo': trafo,
            'Modelo': modelo,
            'RMSE Médio': np.round(np.mean(fold_rmse), 4),
            'MAE Médio': np.round(np.mean(fold_mae), 4)
        })

    return pd.DataFrame(resultados)


In [ ]:
trafos = ['T1']

# resultados_XGB = treinar_e_prever_modelo(df_daily, trafos, modelo='XGB', janela=365)
resultados_XGB_ft = treinar_e_prever_modelo_ft(daily_features, trafos, modelo='XGB', janela=365)
# resultados_LGBM = treinar_e_prever_modelo(daily_features, trafos, modelo='LGBM', janela=365)
